In [ ]:
from matplotlib import cm
from matplotlib import pyplot
from numpy import linalg
from numpy import random
from random import sample
from scipy import signal 
from scipy import stats
from shapely.geometry import Point, Polygon
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_samples
from sklearn.metrics import silhouette_score
from statsmodels.distributions.empirical_distribution import ECDF

import datetime
import foursquare
import geopandas
import math
import numpy
import pandas
import pickle
import pymongo
import time

In [ ]:
tessellation_uri = 'http://dati.comune.milano.it/dataset/806829b9-134b-40cf-b0e0-03e66c4f76d7/resource/3e355dd1-a8b8-483d-ac4d-03a62232ef38/download/ace_maggio_2011.geojson'

# Lantent Activity in Mobility Patterns

In [ ]:
def extract_category(c):
    if not c['categories']:
        return [(c['id'],c['name'])]
    else:
        l = [(c['id'],c['name'])]
        for ca in c['categories']:
            l.extend(extract_category(ca))
        return l

In [ ]:
categories_name = ['Arte e intrattenimento',
 'Istituti superiori e università',
 'Eventi',
 'Cibi',
 'Posti professionali e altri',
 'Locali notturni',
 'All\'aperto & Ricreativi',
 'Negozi e servizi',
 'Viaggi e trasporti',
 'Residenza']

### Pattern di mobilità

In [ ]:
bikemi_dataframe = pandas.DataFrame()

In [ ]:
station_lookup = {}
fwd_lookup, bwd_lookup = {} , {}

In [ ]:
def update_bwd():
    global bwd_lookup
    bwd_lookup = { v:k for k,v in fwd_lookup.items()}

In [ ]:
columns_filter = ['Data_prelievo'
                  , 'Gio_settimana_prelievo'
                  , 'Festivo_feriale_prelievo'
                  , 'Stazione_prelievo'
                  , 'Durata_sec'
                  , 'Data_arrivo'
                  , 'Gio_settimana_arrivo'
                  , 'Festivo_feriale_arrivo'
                  , 'Stazione_arrivo'
                 ]

In [ ]:
#unione dei dataframe
# for yy in range(2015, 2019): 
#     for mm in range(1, 13):
#         try:
#             csv_uri = '/home/datasets/bikemi/{0}/{1:02d} {0}.csv'.format(yy,mm)
#             next_df = pandas.read_csv(csv_uri,
#                            lineterminator ='\r',
#                            encoding = 'iso8859_2',
#                            sep = ';',
#                            parse_dates = ['Data_prelievo','Data_arrivo'],
#                            date_parser = lambda x: datetime.datetime.strptime(x,'%d/%m/%y %H:%M'),
#                            decimal = '.'
#                           )
            
#         except FileNotFoundError:
#             pass
#         else:
#             for _, i in next_df.iterrows():
                
#                 # LOOKUP
#                 a = i['Stazione_prelievo']
#                 b = i['Stazione_arrivo']
                
#                 if a not in station_lookup:
#                     station_lookup[a] = {i['Nome_stazione_prelievo']}
#                 else:
#                     station_lookup[a].add(i['Nome_stazione_prelievo'])
                    
#                 if b not in station_lookup:
#                     station_lookup[b] = {i['Nome_stazione_arrivo']}
#                 else:
#                     station_lookup[b].add(i['Nome_stazione_arrivo'])
                
#             #SAVE
#             bikemi_dataframe = pandas.concat([bikemi_dataframe, next_df[columns_filter]])


# pickle.dump(bikemi_dataframe, open( "archive/bikemi_dataframe.pkl", "wb" ) )
# pickle.dump(station_lookup, open( "archive/station_lookup.pkl", "wb" ) )

In [ ]:
bikemi_dataframe = pickle.load(open('archive/bikemi_dataframe.pkl', 'rb'))

In [ ]:
tmp = pickle.load(open('archive/station_lookup.pkl', 'rb'))
station_lookup = {k:list(v)[0] for k,v in tmp.items()}

In [ ]:
gps = pickle.load(open('/home/datasets/bikemi/station_gps_location.pkl', 'rb'))

### Correzione stazioni 

In [ ]:
# stazioni inutilizzate
for i in list(gps):
    if i not in station_lookup:
        del gps[i]

In [ ]:
# stazioni mancanti
gps[2] = gps[402]
gps[903] = gps[263]

gps[90] = (45.484649, 9.195576)
gps[92] = (45.465589, 9.186123)

In [ ]:
# relookupping
new_gps = []
for i in range(max(gps)+1):
    if i in gps:
        fwd_lookup[i] = len(new_gps)
        new_gps.append(gps[i])
gps = new_gps
update_bwd()

### Correzione POIs 

In [ ]:
# 4square
CLIENT_ID = 'XDNRQIG15QP4PUZEMAGGVBPYHV1WXQMFXXAXZM410USDSTC3'
CLIENT_SECRET = 'M55BW0UXPLUGACETI5ENMSF3WXGLDIENZAHE5VXPPVBYLSWO'
client = foursquare.Foursquare(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
categories = client.venues.categories()

In [ ]:
# lookup macrocateg
category_lookup = {}

In [ ]:
for macro_category in categories['categories']:
    for id_cat, name_cat in extract_category(macro_category):
        category_lookup[id_cat] = (macro_category['id'],macro_category['name'],name_cat)

In [ ]:
# 4square DB
mongo_conn = pymongo.MongoClient("mongodb://marvin.nptlab.di.unimi.it")
six_db = mongo_conn['FoursquarePlacesMilan']
places = six_db['places']

In [ ]:
pois = []

In [ ]:
# pulizia POIS
for i in places.find():
        tmp = {
            'name':i['name'],
            'location':(
                float(i['location']['lat']['$numberDouble']),
                float(i['location']['lng']['$numberDouble'])
            ),
            'categories':{category_lookup[j['id']][0] for j in i['categories']}
        }
        pois.append(tmp)

### Tassellamento 

In [ ]:
raw_aces = geopandas.read_file(tessellation_uri)
vor = {}

In [ ]:
# poligoni delle regioni
for _, r in raw_aces.iterrows():
    lat = r.geometry.exterior.xy[1]
    lng = r.geometry.exterior.xy[0]
    vor[int(r.ACE)-1] = Polygon([(lat[i], lng[i]) for i in range(len(lat))]) # L'ARRAY ACE PARTE DA 1

vor = numpy.array([vor[i] for i in range(len(vor))])

In [ ]:
# relookupping
for g in range(len(gps)):
    for r in range(len(vor)):
        if vor[r].contains(Point(gps[g])):
            fwd_lookup[bwd_lookup[g]] = r
            
del bwd_lookup

In [ ]:
# regioni non vuote
unempty = numpy.array([True]*len(vor))
for r in range(len(vor)):
    unempty[r] = r in fwd_lookup.values()

vor = vor[unempty]

In [ ]:
# relookupping
for k,v in dict(zip((i for i, kept in enumerate(unempty) if kept), range(len(vor)))).items():
    for f,b in fwd_lookup.items():
        if b == k:
            fwd_lookup[f] = v

### TF-IDF e SVD

In [ ]:
# categorie: solo quelle usate, senza lista completa
categories = set()
for i in pois:
    for j in i['categories']: 
        categories.add(j)
categories = list(categories)
categories.sort()

In [ ]:
# frequenze assolute
# abs_frequencies = numpy.zeros([len(vor), len(categories)])
# for i in range(len(vor)):
#     for p in pois:
#         if vor[i].contains(Point(p['location'])):
#             for k in p['categories']:
#                 abs_frequencies[i,categories.index(k)] += 1

# pickle.dump(abs_frequencies, open( "archive/abs_frequencies.pkl", "wb" ) )

In [ ]:
abs_frequencies = pickle.load(open('archive/abs_frequencies.pkl', 'rb'))

In [ ]:
dim_cat = [sum([abs_frequencies[i,j] for i in range(len(vor))]) for j in range(len(categories))] # n° poi per categoria
dim_reg = [sum([abs_frequencies[i,j] for j in range(len(categories))]) for i in range(len(vor))] # n°  poi per regione
num_reg = [len([i for i in range(len(vor)) if abs_frequencies[i,j]]) for j in range(len(categories))] # n° regioni per categoria

In [ ]:
tf_idf = numpy.zeros([len(vor),len(categories)])

In [ ]:
for i in range(len(vor)):
    for j in range(len(categories)):
        tf = dim_cat[j]/dim_reg[i]
        idf = len(vor)/num_reg[j]
        tf_idf[i,j] = tf*math.log(idf)

In [ ]:
u, _, _ = linalg.svd(tf_idf, full_matrices=True)

### Cuboidi

In [ ]:
pickle.dump(fwd_lookup, open( "archive/fwd_lookup.pkl", "wb" ))

In [ ]:
time_gran = 10*60
time_bin = [i for i in range(0,60*60*24*2,time_gran)]

In [ ]:
def i_time_bin(t,f):
    numpy.floor((60*(60*(24*(f)+t.hour)+t.minute)+t.second)/time_gran)

In [ ]:
arriving_cuboid = numpy.zeros([len(vor), len(vor), len(time_bin)])
leaving_cuboid = numpy.zeros([len(vor), len(vor), len(time_bin)])

In [ ]:
# for _, r in bikemi_dataframe.iterrows():
#     src = fwd_lookup[r['Stazione_prelievo']]
#     dst = fwd_lookup[r['Stazione_arrivo']]
    
#     leav_time = i_time_bin(r['Data_prelievo'], r['Festivo_feriale_prelievo'])
#     arr_time = i_time_bin(r['Data_arrivo'], r['Festivo_feriale_arrivo'])
    
#     leaving_cuboid[src, dst, leav_time] += 1
#     arriving_cuboid[src, dst, arr_time] += 1

In [ ]:
arriving_cuboid = pickle.load(open('archive/arriving_cuboid.pkl', 'rb'))
leaving_cuboid = pickle.load(open('archive/leaving_cuboid.pkl', 'rb'))

In [ ]:
docs = {}

In [ ]:
for i in range(len(vor)):
    a = [arriving_cuboid[r,i,t] for r in range(len(vor)) for t in range(len(time_bin))]
    l = [leaving_cuboid[i,r,t] for r in range(len(vor)) for t in range(len(time_bin))]
    m = a + l
    v = u[i]+[1]
    docs[i] = {'v':v, 'm':m}

In [ ]:
vocab = set([docs[i]['m'][n] for i in range(len(vor)) for n in range(len(docs[i]['m']))])
vocab = list(vocab)
vocab.sort()

In [ ]:
tmp = []
for i in range(len(vor)):
    tmp += docs[i]['m']

In [ ]:
fig = pyplot.figure(figsize=(16.5, 6.7))
ax = fig.gca()
eps = 10
yell = 10
y, x = numpy.histogram(tmp, numpy.arange(0, max(tmp)+eps, eps))
ax.plot(x[yell:-1], y[yell:], marker='.',ms=2.5, linestyle='', color='maroon')

In [ ]:
# fig = pyplot.figure(figsize=(16.5, 6.7))
# ax = fig.gca()
# ecdf = ECDF(tmp)
# ax.plot(ecdf.x, ecdf.y, marker='.',ms=0.1, linestyle='--', color='coral')
# ax.set_xlim(0, 40)

In [ ]:
# numpy.mean(tmp), numpy.percentile(tmp, 50), numpy.std(tmp)

###  Probablilità secca

In [ ]:
probs = numpy.array([[abs_frequencies[r,k]/sum(abs_frequencies[r]) for k in range(len(categories))] for r in range(len(vor))])

In [ ]:
for k in range(len(categories)):
    tmp = probs[:,k]
    print((numpy.mean(tmp), numpy.std(tmp)))

In [ ]:
move_probs = numpy.zeros([len(categories),len(categories)])

In [ ]:
# for i, r in bikemi_dataframe.iterrows():
#     src = fwd_lookup[r['Stazione_prelievo']]
#     dst = fwd_lookup[r['Stazione_arrivo']]
#     for ks in range(len(categories)):
#         for kd in range(len(categories)):
#             move_probs[ks,kd] += probs[src,ks]*probs[dst,kd]

### Gibbs Samplig

In [ ]:
# pickle.dump(vor, open('task_data/vor.pkl', 'wb'))
# pickle.dump(vocab, open('task_data/vocab.pkl', 'wb'))
# pickle.dump(categories, open('task_data/categories.pkl', 'wb'))
# pickle.dump(docs, open('task_data/docs.pkl', 'wb'))
# len(vor)*len(time_bin)*2

In [ ]:
b = numpy.zeros([len(categories),len(vocab)]) # wt # b := word topic distribution
z = numpy.zeros([len(vor), len(vor)*len(time_bin)*2]) # ta # z := token topic assignment
th = numpy.zeros([len(vor), len(categories)]) # dt # th := document topic distribution 

In [ ]:
al = 1.5 # varianza rnd_th
eta = 1.5 # varianza rnd_b

In [ ]:
# # random assignment
# for r in range(len(vor)):
#     for n in range(len(docs[r]['m'])):
#         w = vocab.index(docs[r]['m'][n])
#         k = sample(range(len(categories)),1)[0]
        
#         b[k, w] += 1
#         th[r,k] += 1
#         z[r,n] = k

In [ ]:
# # gibbs            
# for iterat in range(1):
#     # catena (vedi Ethen Liu)
#     for r in range(len(vor)):
#         for n in range(len(docs[r]['m'])):
#             w = vocab.index(docs[r]['m'][n])
#             th[r, z[r,n]] -= 1
#             b[z[r,n], w] -= 1
            
#             probs = []
#             for k in range(len(categories)):                
#                 x1 = b[k,w] + eta
#                 x2 = sum(b[k])+ eta*len(vocab)
#                 x = x1/x2
#                 y1 = th[r,k] + al
#                 y2 = sum(th[r])+ al*len(categories)
#                 y = y1/y2
                
#                 probs.append(x/y)
                
#             z[r,n] = random.choice(range(len(categories)), 1, False, p=(probs/sum(probs)))
      
#     # ricalcolo
#     for r in range(len(vor)):
#         for n in range(len(docs[r]['m'])):
#             word_id = vocab.index(docs[r]['m'][n])
#             k = z[r,n]
            
#             b[k, word_id] += 1
#             th[r,k] += 1

In [ ]:
b = pickle.load(open('task_data/b.pkl', 'rb'))
z = pickle.load(open('task_data/z.pkl', 'rb'))
th = pickle.load(open('task_data/th.pkl', 'rb'))

In [ ]:
s = numpy.zeros([len(vor),len(categories)])
e = numpy.zeros([len(categories),len(vocab)])

In [ ]:
for k in range(len(categories)):
    for i in range(len(vocab)):
        e[k,i] = (b[k,i] + eta)/(sum(b[k]) + len(vocab)*eta) # distribuzione categories sulle parole a priori
    for r in range(len(vor)):
        s[r,k] = (th[r,k] + al)/(sum(th[r]) + al*len(categories)) # distribuzione categoria sulle regioni a priori

###  Dirichlet Multinomial Regression - Latent Dirichlet Allocation 

In [ ]:
b = numpy.zeros([len(categories),len(vocab)]) # wt # b := word topic distribution
z = numpy.zeros([len(vor), len(vor)*len(time_bin)*2], int) # ta # z := token topic assignment
th = numpy.zeros([len(vor), len(categories)]) # dt # th := document topic distribution 

In [ ]:
# DRM spec
l = numpy.zeros([len(categories),len(vor)])
a = numpy.zeros([len(vor), len(categories)])

# le nuove parole
m = numpy.zeros([len(vor),len(vor)*len(time_bin)*2])

In [ ]:
for i in range(3):
    for k in range(len(categories)):
        l[k] = random.normal(0.0, 1.0, len(vor)) # qui in vertà ci andrebbe var(s)
        b[k] = random.dirichlet(e[k]) # b : distrubuzione categories sulle parole
        
    for r in range(len(vor)):
        for k in range(len(categories)):
            det = numpy.dot(docs[r]['v'], l[k])
            a[r,k] = numpy.exp(det) # a : parametro per dirichlet
        
        th[r] = random.dirichlet(a[r]) # th : distrubuzione categories sulla regione

        for n in range(len(docs[r]['m'])):
            # print(r,n,end='\r')
            z[r,n] = random.multinomial(1, th[r]).tolist().index(1) # z : category i-regione n-parola = pattern
            docs[r]['m'][n] = vocab[random.multinomial(1, b[z[r,n]]).tolist().index(1)]
    
    # ricalcolo
    tth = numpy.zeros([len(vor),len(categories)])
    bb = numpy.zeros([len(categories),len(vocab)])
    for r in range(len(vor)):
        for n in range(len(docs[r]['m'])):
            w = vocab.index(docs[r]['m'][n])
            k = z[r,n]
            
            bb[k,w] += 1
            tth[r,k] += 1
            
    for k in range(len(categories)):
        for i in range(len(vocab)):
            e[k,i] = (bb[k,i] + eta)/(sum(bb[k]) + len(vocab)*eta)
        for r in range(len(vor)):
            s[r,k] = (tth[r,k] + al)/(sum(tth[r]) + al*len(categories))

In [ ]:
tt = z.copy()
t = [tt[i,j] for i in range(len(tt)) for j in range(len(tt[i]))]

In [ ]:
fig = pyplot.figure(figsize=(16.5, 6.7))
ax = fig.gca()
y, x = numpy.histogram(t, numpy.arange(0, max(t)+2, 1))
ax.plot(x[:-1], y, marker='.',ms=11.0, linestyle='', color='purple')

In [ ]:
categories_name[3]

In [ ]:
t = z[fwd_lookup[334]] # duomo

In [ ]:
fig = pyplot.figure(figsize=(16.5, 6.7))
ax = fig.gca()
y, x = numpy.histogram(t, numpy.arange(min(t), max(t)+2, 1))
ax.plot(x[:-1], y, marker='.',ms=5.5, linestyle='', color='purple')

In [ ]:
reg_a = 1 # Duomo
reg_b = 3 # Cadorna
arr_part_a = 0
hh, mm = 12, 10

categories_name[z[fwd_lookup[reg_a], int((60*hh+mm)/10)*reg_b+(arr_part_a*14400)]]